In [21]:
using Pkg
Pkg.instantiate()
using JuMP
using GLPK
using DataFrames

## Creating tables for the Data

In [61]:
cost_and_effectiveness = DataFrame(Vaccine_Type=[:A, :B, :C, :D], 
                           Cost=[3.2, 3.45, 3.5, 3.3],
                           Effectiveness=[.50,.55,.55,.55])

,Vaccine_Type,Cost,Effectiveness
,Symbol,Float64,Float64
1,A,3.2,0.5
2,B,3.45,0.55
3,C,3.5,0.55
4,D,3.3,0.55


In [62]:
cost_per_vaccine_per_time = DataFrame(Vaccine_Type=[:A, :B, :C, :D],
                                                T₀=[2., 2.3, 3., 2.4],
                                                T₁=[2.45, 2.6, 2.47, 2.6],
                                                T₂=[2.5, 2.8, 2.47, 2.44],
                                                T₃=[2.558, 2.745, 2.87, 2.916],
                                                T₄=[2.3122, 2.2805, 2.3, 2.6244],
                                                T₅=[2.18098, 2.95245, 2.37147, 2.36196 ],
                                                T₆=[2.62882, 2.657205, 2.594323, 2.625764],
                                                T₇=[1.913, 1.913, 1.913, 1.913])

,Vaccine_Type,T₀,T₁,T₂,T₃,T₄,T₅,T₆,T₇
,Symbol,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,A,2.0,2.45,2.5,2.558,2.3122,2.18098,2.62882,1.913
2,B,2.3,2.6,2.8,2.745,2.2805,2.95245,2.6572,1.913
3,C,3.0,2.47,2.47,2.87,2.3,2.37147,2.59432,1.913
4,D,2.4,2.6,2.44,2.916,2.6244,2.36196,2.62576,1.913


In [25]:
population = DataFrame(Age_range=["0-5", "6-18", "19-65","65+"],
                       Population=[19200, 54400, 268400, 58000])

,Age_range,Population
,String,Int64
1,0-5,19200
2,6-18,54400
3,19-65,268400
4,65+,58000


In [52]:
effectiveness_per_population = DataFrame(Age_range=["0-5", "6-18", "19-65","65+"],
                                        A=[.75, .50, .50, .50],
                                        B=[.70,.65, .65, .66],
                                        C=[.60,.60, .65, .60],
                                        D=[.65,.65, .65, .65])

,Age_range,A,B,C,D
,String,Float64,Float64,Float64,Float64
1,0-5,0.75,0.7,0.6,0.65
2,6-18,0.5,0.65,0.6,0.65
3,19-65,0.5,0.65,0.65,0.65
4,65+,0.5,0.66,0.6,0.65


In [53]:
budget_per_time = DataFrame(Time=[:T₀, :T₁, :T₂, :T₃, :T₄, :T₅, :T₆, :T₇ ],
                            Budget=[142850., 142850., 142850., 142850., 142850., 142850., 142850., 142850.])

,Time,Budget
,Symbol,Float64
1,T₀,142850.0
2,T₁,142850.0
3,T₂,142850.0
4,T₃,142850.0
5,T₄,142850.0
6,T₅,142850.0
7,T₆,142850.0
8,T₇,142850.0


In [29]:
function resultant_value(vaccine_type)
    col_ind = findall(x->x==vaccine_type, cost_and_effectiveness[:,names(cost_and_effectiveness)[1]])[1]
    êᵢ = cost_and_effectiveness[:,:Effectiveness][col_ind]
    Σ = sum([e*n for (e,n) in zip(effectiveness_per_population[:, vaccine_type], population[:,:Population])])
    eᵢ = êᵢ * Σ
    return eᵢ
end

resultant_value (generic function with 1 method)

In [30]:
function find_col_index(var, df) 
    ind = findall(x->x==var, df[:,names(df)[1]])[1]
    return ind
end

find_col_index (generic function with 1 method)

## Solving the LP

In [38]:
# V = [resultant_value(vax) for vax in [:A, :B, :C, :D]]
# C = cost_per_vaccine_per_time[:,:T₀]
# B = budget_per_time[:,:Budget][find_col_index(:T₀, budget_per_time)]

# N = length(cost_and_effectiveness[:,:Vaccine_Type])

4

In [41]:
# tvlp = Model(GLPK.Optimizer)
# init = [0 for i=1:N]

# @variable(tvlp, x[i=1:N] >= 0, Int, start = init[i])
# @constraint(tvlp, C'x <= B)
# @objective(tvlp, Max, V'x)
# optimize!(tvlp)
# value.(x)

## TV-LP solver

In [63]:
xs = []
ts = [:T₀, :T₁, :T₂, :T₃, :T₄, :T₅, :T₆, :T₇]
V = [resultant_value(vax) for vax in [:A, :B, :C, :D]]
N = length(cost_and_effectiveness[:,:Vaccine_Type])
init = [0 for i=1:N]


for t in ts 
    C = cost_per_vaccine_per_time[:,t]
    B = budget_per_time[:,:Budget][find_col_index(t, budget_per_time)] 

    tvlp = Model(GLPK.Optimizer) 

    @variable(tvlp, x[i=1:N] >= 500, Int, start = init[i])
    @constraint(tvlp, C'x <= B)
    # @constraint(tvlp, x >=)
    @objective(tvlp, Max, V'x)
    optimize!(tvlp)
    init = value.(x)
    push!(xs, init)
    println("Time step ",t," ",init)
end


Time step T₀ [500.0, 60500.0, 500.0, 500.0]
Time step T₁ [500.0, 508.0, 56277.0, 500.0]
Time step T₂ [500.0, 505.0, 500.0, 56947.0]
Time step T₃ [500.0, 50520.0, 500.0, 500.0]
Time step T₄ [500.0, 61053.0, 500.0, 500.0]
Time step T₅ [500.0, 502.0, 500.0, 58888.0]
Time step T₆ [500.0, 552.0, 500.0, 52850.0]
Time step T₇ [500.0, 73173.0, 500.0, 500.0]


In [58]:
xs

8-element Vector{Any}:
 [500.0, 60500.0, 500.0, 500.0]
 [500.0, 500.0, 56285.0, 500.0]
 [500.0, 500.0, 570.0, 56882.0]
 [500.0, 50516.0, 504.0, 500.0]
 [500.0, 61037.0, 516.0, 500.0]
 [500.0, 500.0, 58656.0, 500.0]
 [500.0, 500.0, 53537.0, 500.0]
 [500.0, 500.0, 73173.0, 500.0]

In [67]:
f=hcat(xs...)
f

4×8 Matrix{Float64}:
   500.0    500.0    500.0    500.0    500.0    500.0    500.0    500.0
 60500.0    508.0    505.0  50520.0  61053.0    502.0    552.0  73173.0
   500.0  56277.0    500.0    500.0    500.0    500.0    500.0    500.0
   500.0    500.0  56947.0    500.0    500.0  58888.0  52850.0    500.0

In [69]:
using PlotlyJS

times = [String(t) for t in ts]

p = plot([
    bar(name="Vaccine A", x=times, y=f[1,:]),
    bar(name="Vaccine B", x=times, y=f[2,:]),
    bar(name="Vaccine C", x=times, y=f[3,:]),
    bar(name="Vaccine D", x=times, y=f[4,:])
])
relayout!(p, barmode="group")
p

data: [
  "bar with fields name, type, x, and y",
  "bar with fields name, type, x, and y",
  "bar with fields name, type, x, and y",
  "bar with fields name, type, x, and y"
]

layout: "layout with fields barmode, margin, and template"